In [1]:
from typing import List

import numpy as np
import tensorflow as tf
import h5py
import os

from tensorflow.python.framework.config import list_physical_devices
from tensorflow.python.keras.utils.np_utils import to_categorical
from keras.src.backend.tensorflow.sparse import zeros_int8
from tensorflow import Tensor

ModuleNotFoundError: No module named 'tensorflow'

In [47]:
#må byttes med der du har lagret filene 256*16 i dette setet
file_path = "C:\\Users\\Kaspar\\ChipWhisperer5_64\\cw\\home\\portable\\chipwhisperer\\jupyter\\courses\\sca101\\TINYAES_test_attack.hdf5"
with h5py.File(file_path, "r") as h5_file:
    plaintext_set = h5_file['data'][:]
    trace_set = h5_file['trace'][:]
    key_set = h5_file['key'][:]
    #sub_byte_out_set = h5_file['sub_byte_out'][:]

In [48]:
trace_set = tf.keras.layers.Rescaling(1. / 127.5, offset=-1)(trace_set)
trace_set = tf.expand_dims(trace_set, axis=-1)

In [49]:
num_traces = len(trace_set)
num_bytes = 16
sub_byte_in_set = np.zeros((num_traces, 16), dtype=np.uint8)

for i in range(num_traces):
    for byte_index in range(num_bytes):
        pts=plaintext_set[i][byte_index] ^ key_set[i,byte_index]
        sub_byte_in_set[i,byte_index] = pts
#check if this works
sub_byte_in_set = sub_byte_in_set

In [50]:
sub_byte_out = np.transpose(sub_byte_out_set, (1, 0))
sub_byte_in = np.transpose(sub_byte_in_set, (1, 0))
key = np.transpose(key_set, (1, 0))
#pts = np.transpose(plaintext_set, (1, 0))

In [56]:
pts.shape

()

In [58]:
num_shards = 16
num_traces_per_shard = 256
file_path = "\\...\\DAT255_SCA\\datasets"

#index to mark start and stop for slicing
start_index = 0
stop_index = num_traces_per_shard
dataset_name = "attack" #change depending on the set you create

#create a new h5py file that will contain the shards
f = h5py.File(f"{dataset_name}.hdf5", "w")

#Loop trough the dataset, creating groups for every key
for i in range(num_shards):
# create a group for each key, the name of the group will be the key in hex. This is refering to the original key
#array, different indexing is needed if you want to use key
    group_name = key_set[start_index].tobytes().hex()
#Create one group representing a shard

    group = f.create_group(group_name, track_order=True)
#oppretter datasettene
#rad, kolonne, steg vi vil ha alle rader, 256 kolonner av gangen. Også vurder å ta to categorical senere i prosessen
#Må treffe på fordelingen her
    group.create_dataset("traces", data = trace_set[start_index:stop_index, :, :])
    group.create_dataset("key", data = key[:, start_index:stop_index])
    group.create_dataset("sub_bytes_in", data = sub_byte_in[:, start_index:stop_index])
    group.create_dataset("sub_bytes_out", data = sub_byte_out[start_index:stop_index: ,])
    group.create_dataset("pts", data = plaintext_set[start_index:stop_index, :])
#må også ha trace slik at den kan hentes
    start_index += num_traces_per_shard
    stop_index += num_traces_per_shard
f.close()

In [57]:
f.close()

In [60]:
tf.config.list_physical_devices("GPU")

[]

In [62]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0
